Tarea 1 - PDM - Vicente Espinoza - Marcelo Vargas

In [1]:
import pandas as pd
import kshingle as ks
from datasketch import MinHash, MinHashLSH
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer as TbWD
import re

In [9]:
def buscar_parecidos(username, k, dframe, lsh):
    # Input: Username (str)

    # Output: Lista de Username de usuarios que han escrito tweets similares

    listindex = list(dframe[dframe['screen_name'] == username].index)

    users_similares= dict()

    for indexuser in listindex:
        minhash = dframe.loc[indexuser]['shingles']
        for user_id in lsh.query(minhash):
            similar_name = dframe.loc[user_id]['screen_name']
            
            if similar_name != username:
                users_similares[similar_name] = users_similares.get(similar_name, 0) + 1

    finallist = [(name,value) for name,value in users_similares.items()]

    finallist.sort(key=lambda x: x[1], reverse=True)

    return finallist[:k]

In [2]:
def eliminar_emojis(texto):
    # Expresión regular para eliminar emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticones
                               u"\U0001F300-\U0001F5FF"  # símbolos y pictogramas
                               u"\U0001F680-\U0001F6FF"  # transporte y símbolos de mapa
                               u"\U0001F1E0-\U0001F1FF"  # banderas de países
                               u"\U00002500-\U00002BEF"  # caracteres chinos, japoneses y coreanos (CJK)
                               u"\U00002702-\U000027B0"  # símbolos de negocio
                               u"\U00002702-\U000027B0"  # flechas y símbolos diversos
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    
    # Eliminar emojis
    texto_sin_emojis = emoji_pattern.sub(r'', texto)
    
    # Preservar letras con tildes, la letra "ñ" y símbolos específicos
    texto_limpio = re.sub(r'[^a-zA-Zá-úÁ-ÚñÑ@/: ,.;-_]+', '', texto_sin_emojis)
    
    return texto_limpio

def set_to_minhash(set, minhash):
    for shingle in set:
        minhash.update(shingle.encode('utf-8'))

    return minhash

def buscar_parecidos(username, k, dframe, lsh):
    # Input: Username (str)

    # Output: Lista de Username de usuarios que han escrito tweets similares

    listindex = list(dframe[dframe['screen_name'] == username].index)

    users_similares= dict()

    for indexuser in listindex:
        minhash = dframe.loc[indexuser]['shingles']
        for user_id in lsh.query(minhash):
            similar_name = dframe.loc[user_id]['screen_name']
            
            if similar_name != username:
                users_similares[similar_name] = users_similares.get(similar_name, 0) + 1

    finallist = [(name,value) for name,value in users_similares.items()]

    finallist.sort(key=lambda x: x[1], reverse=True)

    return finallist[:k]

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
stopwords_spanish = (stopwords.words('spanish'))

df = pd.read_csv('tweets_2022_abril_junio.csv', usecols = ['id', 'screen_name', 'text'], index_col = "id")
df = df[~df.index.duplicated(keep='first')]
df = df[df["text"].str[:2] != "RT"]

df['text'] = df['text'].apply(lambda x: TbWD().detokenize([word for word in eliminar_emojis(x.lower()).split(' ') if word not in stopwords_spanish]))
df['shingles'] = df["text"].apply(lambda x: ks.shingleset_list(x, [3]))

df["shingles"] = df["shingles"].apply(lambda x: set_to_minhash(x, MinHash(128)))


In [5]:
lsh = MinHashLSH(threshold = 0.5, num_perm = 128)
for ind in df.index:
    lsh.insert(ind, df["shingles"][ind])

In [10]:
buscar_parecidos('MacaSimplemente', 6, df, lsh)

[('pita09443405', 441),
 ('Arnold73020321', 289),
 ('Sergio_Valech', 172),
 ('JCCONTACTOSUR', 150),
 ('LuzTamu', 142),
 ('karinwinder', 139)]